# Initial usage of meshmagick
This notebook provides a short intro to meshmagick. Checkout the [API documentation](https://lheea.github.io/meshmagick/api/index.html) 

In [1]:
import os
from meshmagick import mmio, mesh, hydrostatics

## 1. Load the geometry file
Here we are using `os.getcwd()` because `mmio.load_STL` requires absolute paths.

In [2]:
path = os.getcwd() + '/models/'
file = "box.stl"

V, F = mmio.load_STL(path + file)

## 2. Create the mesh object

In [3]:
mymesh = mesh.Mesh(V, F)
print("Bounding box:", mymesh.squared_axis_aligned_bbox)

Bounding box: (-3.0, 3.0, -3.0, 3.0, -3.0, 3.0)


We can visualize the geometry by calling `Mesh.show()`:

In [4]:
mymesh.show()

Meshmagick provides several functions to fix geometry issues:

In [5]:
# mymesh.merge_duplicates()
# mymesh.triangulate_quadrangles()
# mymesh.heal_triangles()
# mymesh.remove_unused_vertices()
# mymesh.remove_degenerated_faces()
# mymesh.heal_normals()
# mymesh.heal_mesh()

And we can check if the mesh is closed too:

In [6]:
print("Is it closed?", mymesh.is_mesh_closed())

Is it closed? True


And inspect the mesh quality

In [7]:
mymesh.print_quality()


Triangle quality of the mesh (12 elements):

 Area Ratio:
    range: 0.24  -  3.6
    average: 1.68  , standard deviation: 1.47592
 Edge Ratio:
    range: 3.16228  -  15.0333
    average: 7.76486  , standard deviation: 5.43124
 Aspect Ratio:
    range: 2.17941  -  8.97843
    average: 4.80843  , standard deviation: 3.11449
 Radius Ratio:
    range: 1.88743  -  7.77555
    average: 4.16423  , standard deviation: 2.69723
 Frobenius Norm:
    range: 1.9245  -  8.69874
    average: 4.54182  , standard deviation: 3.10429
 Minimal Angle:
    range: 3.81407  -  18.4349
    average: 11.1863  , standard deviation: 6.23503

Quadrilateral quality of the mesh (0 elements):

 Area Ratio:
    range: 0  -  0
    average: 0  , standard deviation: 0
 Edge Ratio:
    range: 0  -  0
    average: 0  , standard deviation: 0
 Aspect Ratio:
    range: 0  -  0
    average: 0  , standard deviation: 0
 Radius Ratio:
    range: 0  -  0
    average: 0  , standard deviation: 0
 Average Frobenius Norm:
    range: 

## Hydrostatics Solver
With the Mesh loaded we can create a Hydrostatics solver and set its **c.o.g.** (center of gravity) location ($x, y, z$ in meters, relative to the model origin).

We can also add a displacment (in TONS) and call `Hydrostatics.equilibrate()` to run the solver, then print the results and visualize:

In [8]:
mymesh = mesh.Mesh(V, F)
hs_solver = hydrostatics.Hydrostatics(mymesh, verbose=True)

disp = 0.5
cog = (2, 0.1, 0)

hs_solver.gravity_center = cog
hs_solver.mass = disp
hs_solver.equilibrate(init_disp=True)

print(hs_solver.get_hydrostatic_report())

hs_solver.show()

First placing the mesh at the target displacement
-------------------------------------------------

Complying with a displacement of 0.500 tons
----------------------------------------------
	-> Convergence obtained after 2 iterations
	-> Mesh has been translated in z by: 0.132117

Computing equilibrium from initial condition.
----------------------------------------------
Stable equilibrium reached after 6 iterations and 0 random restart


	GRAVITY ACCELERATION (M/S**2)----------->   9.81
	DENSITY OF WATER (KG/M**3)-------------->   1023.0

	WATERPLANE AREA (M**2)------------------>   3.4
	WATERPLANE CENTER (M)------------------->   1.556     0.062    
	WET AREA (M**2)------------------------->   4.6
	DISPLACEMENT VOLUME (M**3)-------------->   0.489
	DISPLACEMENT MASS (TONS)---------------->   0.500
	BUOYANCY CENTER (M)--------------------->  -0.000     0.000    -0.098    
	CENTER OF GRAVITY (M)------------------->   0.000     0.000     0.005    

	DRAUGHT (M)-----------------------